In [ ]:
# Import Dependencies
from dotenv import load_dotenv
import os
import base64
from requests import post,get
import json
import requests
import pandas as pd

# Load Client Secret and Credentials
load_dotenv()

# Save Client Secret and Credentials into variables
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

# Function to get token
def get_token():
    auth_string = client_id + ":" + client_secret
    auth_bytes = auth_string.encode('utf-8')
    auth_base64 = str(base64.b64encode(auth_bytes), "utf-8")

    url = "https://accounts.spotify.com/api/token"
    headers = {
        "Authorization" : "Basic "+ auth_base64,
        "Content-Type": "application/x-www-form-urlencoded"
    }
    data = {"grant_type": "client_credentials"}
    result = post(url, headers=headers, data=data)
    json_result = json.loads(result.content)
    token = json_result["access_token"]
    return token

# Function to get header
def get_auth_header(token):
    return {"Authorization": "Bearer " + token}

# Function to search for artist
def search_for_artist(token, artist_name):
    url = "https://api.spotify.com/v1/search"
    headers = get_auth_header(token)
    query = f"?q={artist_name}&type=artist&limit=1"
    query_url = url + query
    result= get(query_url, headers = headers)
    json_result = json.loads(result.content)["artists"]["items"]
    if len(json_result) == 0:
        print("No artist with this name exists")
        return None
    
    return json_result[0]

# Function to get song recommendation
def get_songs_recommendations(token, artist_id, min_popularity, max_popularity):
    url = f"https://api.spotify.com/v1/recommendations?limit=100&market=US&seed_artists={artist_id}&min_popularity={min_popularity}&max_popularity={max_popularity}"
    headers = get_auth_header(token)
    result = get(url, headers= headers)
    if result.status_code ==200:
        json_result=json.loads(result.content)
        return json_result['tracks']
    else:
        print(f"Failed to fetch recommendations. Status code: {result.status_code}")
        return None


In [ ]:
# Load in artist id
artist_id_df = pd.read_csv('artist_id.csv')

In [ ]:
# Save artist id as series
artist_id_list = artist_id_df['artist_id']

In [ ]:
# Break list into 3 parts
artist_chunk1 = artist_id_list[0:1000]
artist_chunk2 = artist_id_list[1000:2000]
artist_chunk3 = artist_id_list[2000:3000]

In [ ]:
# Get token and save recommendations for artists in first chunk
token = get_token()
recommended_song_list1 = []
recommended_song_id1 = []

for artist_id in artist_chunk1:
    song_low =  get_songs_recommendations(token, artist_id, 0, 33)
    song_med =  get_songs_recommendations(token, artist_id, 34, 66)
    song_high =  get_songs_recommendations(token, artist_id, 67, 100)
    songs = []
    songs.extend(song_low)
    songs.extend(song_med)
    songs.extend(song_high)
    for song in songs:
        # print(song['name'])
        # print(song['id'])
        recommended_song_id1.append(song['id'])
        for artist in song['artists']:
        # print(artist['name'])
        # print("------------")
            dict = {
                'name' : song['name'],
                'artist' : artist['name'],
                'id': song['id'],
                'popularity' : song['popularity'],

            }
            recommended_song_list1.append(dict)


In [ ]:
# Get token and save recommendations for artists in second chunk
token = get_token()
recommended_song_list2 = []
recommended_song_id2 = []

for artist_id in artist_chunk2:
    song_low =  get_songs_recommendations(token, artist_id, 0, 33)
    song_med =  get_songs_recommendations(token, artist_id, 34, 66)
    song_high =  get_songs_recommendations(token, artist_id, 67, 100)
    songs = []
    songs.extend(song_low)
    songs.extend(song_med)
    songs.extend(song_high)
    for song in songs:
        # print(song['name'])
        # print(song['id'])
        recommended_song_id2.append(song['id'])
        for artist in song['artists']:
        # print(artist['name'])
        # print("------------")
            dict = {
                'name' : song['name'],
                'artist' : artist['name'],
                'id': song['id'],
                'popularity' : song['popularity'],

            }
            recommended_song_list2.append(dict)

In [ ]:
# Get token and save recommendations for artists in third chunk
token = get_token()
recommended_song_list3 = []
recommended_song_id3 = []


for artist_id in artist_chunk3:
    song_low =  get_songs_recommendations(token, artist_id, 0, 33)
    song_med =  get_songs_recommendations(token, artist_id, 34, 66)
    song_high =  get_songs_recommendations(token, artist_id, 67, 100)
    songs = []
    songs.extend(song_low)
    songs.extend(song_med)
    songs.extend(song_high)
    for song in songs:
        # print(song['name'])
        # print(song['id'])
        recommended_song_id3.append(song['id'])
        for artist in song['artists']:
        # print(artist['name'])
        # print("------------")
            dict = {
                'name' : song['name'],
                'artist' : artist['name'],
                'id': song['id'],
                'popularity' : song['popularity'],

            }
            recommended_song_list3.append(dict)

In [ ]:
# Compile into 
recommended_song_list = []
recommended_song_list.extend(recommended_song_list1)
recommended_song_list.extend(recommended_song_list2)
recommended_song_list.extend(recommended_song_list3)

In [ ]:
df_recommended_songs = pd.DataFrame(recommended_song_list)

In [ ]:
df_recommended_songs = df_recommended_songs.drop_duplicates(subset = ['name']).reset_index(drop = True)
df_recommended_songs

In [ ]:
df_recommended_songs.to_csv('recommended_song.csv')